In [1]:
import os
import csv
import pandas as pd
import numpy as np

import re
from stop_words import get_stop_words
import unicodedata
from num2words import num2words
import nltk
from nltk.stem import WordNetLemmatizer

Tras hacer toda la limpieza recordé lo descompensadas que estaban las reviews, por ello, opté por coger las 5000 primeras entradas con sentimiento positivo y las 5000 primeras con sentimiento negativo.

In [2]:
dataset_path = '../dataset'

videogames_json = 'reviews_Video_Games_5.json.gz'
pet_json = 'reviews_Pet_Supplies_5.json.gz'

videogames = pd.read_json(os.path.join(dataset_path, videogames_json), lines=True, compression='gzip')
pet = pd.read_json(os.path.join(dataset_path, pet_json), lines=True, compression='gzip')


In [3]:
# Preparar los datos:
# - Transformamos las puntuaciones de estrellas a positivas y negativas.
# - Utilizamos la misma cantidad de reviews positivas y negativas para balancear el dataset, 
#   que como ya hemos visto anteriormente esta bastante desbalanceado.
def prepare_data(df, x):
    df = df.dropna(subset=['reviewText', 'overall'])[['reviewText', 'overall']]
    df["overall"].replace({1: 0, 2: 0, 3: 0, 4: 1, 5: 1}, inplace=True)
    
    positive = df[df["overall"] > 0]
    negative = df[df["overall"] < 1]
    
    positive = positive[0:x]
    negative = negative[0:x]
    
    df = pd.concat([positive, negative], axis=0)
    
    dataframe = df.sample(frac=1).reset_index(drop=True)
    
    return dataframe

# Pasamos todo a minúsculas.
def lower(text):
    text = text.lower()
    
    return text

# Eliminamos caracteres no deseados y números.
def clean_characters(text):
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[0-9]+', ' ', text)
    
    return text 

# Eliminamos las stopwords con la lista que nos proporciona la librería
def eliminate_stopwords(text):
    sw_list = get_stop_words('en')
    text = ' '.join([word for word in text.split() if word not in sw_list])
    
    return text

# Lematizamos.
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text
# Unimos las funciones anteriores
def clean_text(text):
    text = lower(text)
    text = clean_characters(text)
    text = eliminate_stopwords(text)
    text = lemmatization(text)

    return text



# Juntamos las funciones que trabajan sobre cada una de las columnas
def cleaning(df):
    reviews = []
    
    for text in df.reviewText:
        reviews.append(clean_text(text))
    
    sentiment = []
    for item in df.overall:
        if item > 0:
            sentiment.append(1)
        else:
            sentiment.append(0)
    
    df = pd.DataFrame({
    'review': reviews,
    'sentiment': sentiment
    })
    df.dropna(subset=['review', 'sentiment'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df



# Juntamos todas las funciones
def transform_df_to_text(df, x):
    a = prepare_data(df, x)
    df = cleaning(a)
    return df

In [4]:
clean_pet = transform_df_to_text(pet, 50)
clean_videogames = transform_df_to_text(videogames, 50)

In [5]:
clean_pet[0:3]

,review,sentiment
0,using supplement year joint somewhat tolerable...,1
1,gotten just volume two given five star since g...,0
2,fairly disappointed pleased w stand tent short...,0
